In [4]:
import pandas as pd
import numpy as np

# Computations of emissions from Commmuting (to and from work)
* Calculations based on results from an internal commuting survey 
* Responses 112 of 127 
* Survey frm May 2020 (asked what the respondents usually does (not during Covid-19 lockdown)
* About 16 employees in Oslo, and 111 in Trondheim. Figures calculates as if all employees are working in Trondheim

In [5]:
survey = ['Walk/run','Public Transport','Bike','Car (EL)','Car (Fuel)']
survey = pd.DataFrame(survey, columns = ['Means of transportation'])


In [6]:
# Manually input data
survey['Percent'] = [11.6,58.9,18.8,6.3,4.5]
survey['Respondents'] = [112*0.116,112*0.589,112*0.188,112*0.063,112*0.045]
survey['Respondents'] = survey['Respondents'].round(decimals=0)

#### Scale figures to map all emplopyees

In [8]:
Employees = 131 # by end of 2020 
survey['Number'] = [survey.Percent[0]*Employees/100, 
                                survey.Percent[1]*Employees/100,
                                survey.Percent[2]*Employees/100,
                                survey.Percent[3]*Employees/100,
                                survey.Percent[4]*Employees/100]
survey['Number'] = survey['Number'].round(decimals=0)
survey

,Means of transportation,Percent,Respondents,Number
0,Walk/run,11.6,13.0,15.0
1,Public Transport,58.9,66.0,77.0
2,Bike,18.8,21.0,25.0
3,Car (EL),6.3,7.0,8.0
4,Car (Fuel),4.5,5.0,6.0


# Emission from buss transport

### Average travel distances for all commuting in Norway split by mean of transportatioin and for different cities 
Source: https://www.vegvesen.no/_attachment/2674990/binary/1361215?fast_title=N%C3%B8kkelrapport+Reisevaneunders%C3%B8kelsen+2018+-+november+2019.PDF (side 17)

In [9]:
distance_mean = 16.23

### Emission / kilometer from AtB (local buss company) 
Source: https://www.atb.no/getfile.php/1383892-1593157917/Rapporter/AtB_a%CC%8Arsrapport_2019.pdf (page 43)

In [12]:
# Assume that all public transportation in by buss (this excludes ferry / boat)
total_distance_2020 = 32939337
total_emission_2020 = 24547000
Co2_per_km = total_emission_2020 / total_distance_2020
print('Kg CO2 per km = ', Co2_per_km)

Kg CO2 per km =  0.7452183995081625


#### Total distance for all employees

In [10]:
total_travel_day = survey.Number[1]*distance_mean
total_travel_week = survey.Number[1]*distance_mean*5
total_travel_year = survey.Number[1]*distance_mean*230
travel = {'Time unit':['Day','Week','Year'], 'Distanse kjørt (km)':[total_travel_day,total_travel_week,total_travel_year]}
travel = pd.DataFrame(travel)
travel['Antall dager']  = 1,5,230
travel 

,Time unit,Distanse kjørt (km),Antall dager
0,Day,1249.71,1
1,Week,6248.55,5
2,Year,287433.30,230


### Wagon size  og capacity utilization

In [13]:
url = 'https://www.ssb.no/statbank/sq/10037029'

AtB_data = pd.read_excel(url, skiprows=2)
AtB_data = AtB_data[:2]
AtB_data = AtB_data.drop('Unnamed: 0', axis=1)
AtB_data = AtB_data.drop(0)
AtB_data.rename(columns={'Vognstørrelse (seter)': 'Wagon size',
                         'Kapasitetsutnyttelse (prosent)':'Capcity utilization'}, inplace = True)
AtB_data

,Wagon size,Capcity utilization
1,47.0,53.0


In [15]:
# Assume 100% capcity utilization due to travel during rush hours
AtB_data['Capcity utilization'][1] = 100
AtB_data

,Wagon size,Capcity utilization
1,47.0,100.0


#### Number of traveler per bus

In [16]:
no_travelers_bus = AtB_data['Wagon size'][1]
no_travelers_bus

47.0

### Empission per traveller

In [18]:
Co2_per_km_per_traveller = Co2_per_km / no_travellers_bus
print('Co2 emission per km per traveler =', Co2_per_km_per_traveler)

Co2 emission per km per traveller = 0.015855710627833246


#### Total emission during one week (kg Co2)

In [12]:
emissionUtslipp_buss_uke_kg = Total_reise_uke * Co2_per_km_per_reisende
Utslipp_buss_uke_kg.round(decimals=4)

99.0752

#### Total utslipp i løpet av et år (tonn Co2)


In [13]:
# Antar 230 arbeidsdager / 46 arbeidsuker i året 
Utslipp_buss_år = Utslipp_buss_uke_kg * 46
Utslipp_buss_år_tonn = Utslipp_buss_år/1000
Utslipp_buss_år_tonn = Utslipp_buss_år_tonn.round(decimals=2)
print('Utslipp fra buss i året =', Utslipp_buss_år_tonn, 'tCO2e')

Utslipp fra buss i året = 4.56 tCO2e


## Emission from driving (non-electric wehicles)

In [14]:
kilde = 'https://www.framtiden.no/gronne-tips/reise-og-transport/klimagassutslippet-fra-ulike-reisemater.html'
kilde2 ='https://www.ssb.no/natur-og-miljo/artikler-og-publikasjoner/hva-pavirker-utslipp-til-luft-fra-veitrafikk?tabell=318157'
kilde3 = 'https://www.ssb.no/statbank/list/bilreg/'

In [15]:
url = 'https://www.ssb.no/statbank/sq/10037343'
cars = pd.read_excel(url,skiprows=2)
cars.rename(columns={'Unnamed: 2':'Type bil'},inplace=True)
cars = cars[['Type bil', 'Personbiler']]
cars = cars[:3]
cars

,Type bil,Personbiler
0,NaN,2019.0
1,Bensin,29686.0
2,Diesel,36077.0


In [16]:
Antall_bensin = cars.Personbiler[1]
Antall_diesel = cars.Personbiler[2]
Antall_totalt = cars.Personbiler[1] + cars.Personbiler[2]

In [17]:
Utslipp_per_kilometer = (0.156*Antall_bensin + 0.130*Antall_diesel)/ Antall_totalt

#### Average travel length and number of travellers

In [18]:
# Avg. travel length 
Distanse_Gjennomsnitt = 15.69
Antall_reisende = spørreundersøkelse.Antall[4]
Antall_reisende #no of travellers by car

6.0

#### Emission from driving 

In [19]:
Utslipp_bil_år = Distanse_Gjennomsnitt*Utslipp_per_kilometer*Antall_reisende*230
Utslipp_bil_år_tonn = (Utslipp_bil_år/1000).round(decimals=2)
print('Utslipp fra bilkjøring i året =', Utslipp_bil_år_tonn, 'tCO2')

Utslipp fra bilkjøring i året = 3.07 tCO2


## Emission from electric wehicles (EL)

### Electricy consumption and emission per kilometer 

In [20]:
# Source: https://www.fjordkraft.no/strom/stromforbruk/elbil/
kWh_EV = 0.2
# Source: https://www.nve.no/energibruk-effektivisering-og-teknologier/energibruk/hvor-kommer-strommen-fra/?ref=mainmenu
emissions_kWh = 0.017

In [21]:
# Same travel distance as for other car travels
Distanse_Gjennomsnitt = 15.69

In [22]:
Antall_reisende_EL = spørreundersøkelse.Antall[3]
print(Antall_reisende_EL)

8.0


In [23]:
Utslipp_ELbil_år = Distanse_Gjennomsnitt*emissions_kWh*kWh_EV*Antall_reisende_EL*230
Utslipp_ELbil_år_tonn = (Utslipp_ELbil_år/1000).round(decimals=2)
print('Utslipp fra EL bilkjøring i året =', Utslipp_ELbil_år_tonn, 'tCO2')

Utslipp fra EL bilkjøring i året = 0.1 tCO2


# Total

In [24]:
df_verdict_commute = pd.DataFrame(index = ["Bil", "Buss",'EL-bil'], columns=["Distanse [pkm]", "Utslipp, CO2 [tonn]"], dtype=int)
df_verdict_commute["Distanse [pkm]"] = [int(Distanse_Gjennomsnitt*Antall_reisende*230), int(Total_reise_år),int(Distanse_Gjennomsnitt*Antall_reisende_EL*230)]
df_verdict_commute["Utslipp, CO2 [tonn]"] = [round(Utslipp_bil_år_tonn, 2), round(Utslipp_buss_år_tonn, 2),round(Utslipp_ELbil_år_tonn,2)]

In [25]:
print("Employee commuting")
print("df_verdict_commute:")
display(df_verdict_commute)

Employee commuting
df_verdict_commute:


,Distanse [pkm],"Utslipp, CO2 [tonn]"
Bil,21652,3.07
Buss,287433,4.56
EL-bil,28869,0.10
